In [1]:
import os
from tryRAG.framework import RAGFramework

/home/yasaisen/anaconda3/envs/MVLM_RAG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
doc_path = os.path.join(os.getcwd(), "../..", "ref_docs", "eecs_20250606_text_bs_rewritten.jsonl")

cfg = {
    "lm_model_name": "../../gemma-3-4b-it",
    "emb_model_name": "all-MiniLM-L6-v2",
    "doc_path": doc_path, 
    # "idx_path": None,
    "device": "cuda",
}

In [ ]:
rag = RAGFramework.from_config(cfg)

question = 'Who is Lee Julian Purnell'
question = 'For how long has Berkeley EECS been at the forefront of research? Available options:\n    (A) Less than a decade\n    (B) Over a quarter of a century\n    (C) Over half a century\n    (D) Over a century'

response = rag.ask(question)

print(response['response'])

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.01it/s]


Successfully loaded 4809 document chunks
Save index to ./ref_idx/
RAGFramework initialized

Lee Julian Purnell is the first known Black student to graduate from the EECS department at UC Berkeley. He was born in Washington, D.C. in 1896, graduated from Berkeley High in 1915, earned a B.A. from Cal in 1919, a B.S. in Electrical Engineering from MIT in 1921, and an M.S. in Electrical Engineering from Berkeley in 1929.
